###Repository and Data Setup

In [0]:
#Setup Repo
!git clone https://github.com/bivasmaiti26/siamese-mask-rcnn.git
#Get Imagenet Weights
%cd /content/siamese-mask-rcnn
%mkdir checkpoints
%cd /content/siamese-mask-rcnn/checkpoints
!wget https://github.com/bethgelab/siamese-mask-rcnn/releases/download/v0.1/imagenet_687.h5
%cd /content/siamese-mask-rcnn/data
!git clone https://github.com/bivasmaiti26/cocoapi.git


In [0]:
#Setup Data (for Google Colab)
%cd /content/
from colab_utils import GoogleDrive
drive = GoogleDrive()
drive.download_file('ycb_train.zip', 'ycb_train.zip')
drive.download_file('ycb_val.zip', 'ycb_val.zip')
!unzip ycb_train.zip > train.out
!unzip ycb_val.zip > val.out


In [0]:
#Installation of correct versions of stuff
%cd /content/siamese-mask-rcnn/
!sudo pip uninstall scikit_image
!sudo pip uninstall keras
!pip uninstall pycocotools
! sudo pip install numpy==1.14.1 cython scikit_image==0.13.1 keras==2.1.6 h5py imgaug opencv_python
! (cd data/cocoapi/PythonAPI/ && sudo make install)

**Restart Runtime after all installations**

###Imports

In [0]:
%cd /content/siamese-mask-rcnn/
%load_ext autoreload
%autoreload 2
%matplotlib inline
#%load_ext line_profiler

import tensorflow as tf
tf.logging.set_verbosity(tf.logging.INFO)
sess_config = tf.ConfigProto()

import sys
import os

COCO_DATA = 'data/coco'
MASK_RCNN_MODEL_PATH = 'lib/Mask_RCNN/'

if MASK_RCNN_MODEL_PATH not in sys.path:
    sys.path.append(MASK_RCNN_MODEL_PATH)
    
from samples.ycb import ycb
from mrcnn import utils
from mrcnn import model as modellib
from mrcnn import visualize
    
from lib import utils as siamese_utils
from lib import model as siamese_model
from lib import config as siamese_config
   
import time
import datetime
import random
import numpy as np
import skimage.io
import imgaug
import pickle
import matplotlib.pyplot as plt
from collections import OrderedDict

# Root directory of the project
ROOT_DIR = os.getcwd()

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")
%cd /content/siamese-mask-rcnn/

In [0]:
exclude_classes = [1,5,9,14]
train_classes = np.array(range(1,21))[np.array([i not in exclude_classes for i in range(1,21)])]


###Dataset Setup

In [0]:
index = 1 #experiment number

In [0]:
#Put link to dataset
%cd /content/siamese-mask-rcnn/
!ln -s /content/ycb_train ycb_train
!ln -s /content/ycb_val ycb_val

In [0]:
# Load YCB/train dataset- 
ycb_train = siamese_utils.IndexedYCBDataset('train')
ycb_train.load_ycb()
ycb_train.prepare()
ycb_train.build_indices()
ycb_train.ACTIVE_CLASSES = train_classes


# Load YCB/val dataset
ycb_val = siamese_utils.IndexedYCBDataset('val')
ycb_val.load_ycb(100)
ycb_val.prepare()
ycb_val.build_indices()
ycb_val.ACTIVE_CLASSES = train_classes

###Training Configuration

In [0]:
class TrainConfig(siamese_config.Config):
    # Set batch size to 1 since we'll be running inference on
    # one image at a time. Batch size = GPU_COUNT * IMAGES_PER_GPU
    GPU_COUNT = 4
    IMAGES_PER_GPU = 3
    NUM_CLASSES = 1 + 1
    NAME = 'parallel_coco'
    EXPERIMENT = 'i{}'.format(index)
    CHECKPOINT_DIR = '../checkpoints/'
    # Reduced image sizes
    TARGET_MAX_DIM = 192
    TARGET_MIN_DIM = 150
    IMAGE_MIN_DIM = 800
    IMAGE_MAX_DIM = 1024
    # Reduce model size
    FPN_CLASSIF_FC_LAYERS_SIZE = 1024
    FPN_FEATUREMAPS = 256
    # Reduce number of rois at all stages
    RPN_ANCHOR_STRIDE = 1
    RPN_TRAIN_ANCHORS_PER_IMAGE = 256
    POST_NMS_ROIS_TRAINING = 2000
    POST_NMS_ROIS_INFERENCE = 1000
    TRAIN_ROIS_PER_IMAGE = 200
    DETECTION_MAX_INSTANCES = 100
    MAX_GT_INSTANCES = 100
    # Adapt NMS Threshold
    DETECTION_NMS_THRESHOLD = 0.5
    # Adapt loss weights
    LOSS_WEIGHTS = {'rpn_class_loss': 2.0, 
                    'rpn_bbox_loss': 0.1, 
                    'mrcnn_class_loss': 2.0, 
                    'mrcnn_bbox_loss': 0.5, 
                    'mrcnn_mask_loss': 1.0}
    
config = TrainConfig()
config.display()

In [0]:
# Create model object in inference mode.
model = siamese_model.SiameseMaskRCNN(mode="training", model_dir=MODEL_DIR, config=config)

In [0]:
train_schedule = OrderedDict()
train_schedule[1] = {"learning_rate": config.LEARNING_RATE, "layers": "heads"}
train_schedule[120] = {"learning_rate": config.LEARNING_RATE, "layers": "all"}
train_schedule[160] = {"learning_rate": config.LEARNING_RATE/10, "layers": "all"}

###Load Model

In [0]:
%cd /content/siamese-mask-rcnn/lib/
# Load weights trained on Imagenet
try: 
    model.load_latest_checkpoint(train_schedule=train_schedule)
except:
    model.load_imagenet_weights(pretraining='imagenet-687')

###Train

In [0]:
%cd /content/siamese-mask-rcnn/
for epochs, parameters in train_schedule.items():
    print("")
    print("training layers {} until epoch {} with learning_rate {}".format(parameters["layers"], 
                                                                          epochs, 
                                                                          parameters["learning_rate"]))
    model.train(ycb_train, ycb_val, 
                learning_rate=parameters["learning_rate"], 
                epochs=epochs, 
                layers=parameters["layers"])